# Using Linear Regression to Predict House Sale Prices
---
Written by Neil Mackenzie

# Introduction
This guided project was completed following a Linear Regression [course](https://www.dataquest.io/course/linear-algebra-for-machine-learning/) on Dataquest.io

The aim of this project is to practice the implementation of linear regresion to predict the price of houses in Ames, Iowa, United States.

The original paper written by Dean De Cock presenting this dataset can be found [here](https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627). The xls version of the dataset can be downloaded directly from [this](http://www.amstat.org/publications/jse/v19n3/decock/AmesHousing.xls) link and information about the columns of the dataset can be found on [this](https://s3.amazonaws.com/dq-content/307/data_description.txt) page. Note the xls file was saved as a csv file for this project

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from IPython.display import display, HTML
pd.options.display.max_columns = 999
pd.options.display.max_rows = 10

#Read dataset
housing_data = pd.read_csv('AmesHousing.csv', encoding = 'utf-8')
#Preview first 5 rows of dataset
housing_data.head(5)

# Data Cleaning & Feature Engineering
The feature columns will be updated in the cells below to handle missing values, transform text and numerical columns and create new columns that reveal valuable information about each house.

The strategies for cleaning the dataset and creating new columns are tested below before being implemented into new functions that will carry execute the operations. The operations are outlined below and tested in the cells that follow.
1. Drop missing data
2. Drop columns containing information that is not valuable to an ML model
3. Feature engineering to identify and generate useful information
4. Drop columns that are no longer required or leak information about the final sale since this would "cheat" the model

# Test operations
---
The strategies for cleaning the dataset are tested below before being implemented into an ML model. The four steps used to clean and prepare the dataset are:
1. Drop missing data
2. Drop columns that will not be useful for an ML model
3. Feature engineering to derive new columns that reveal valuable information
4. Drop columns that leak/reveal information about the sale price of the house since this information will bias the ML model

## 1.  Drop missing data

Missing data will be handled differently depending on the amount of data missing in the column and the type of data stored in the column. The strategy for cleaning columns is described below:
1. All columns
    - Drop column if > 5% of data in column is missing
2. Numerical columns 
    - Columns containing less than 5% missing data will have missing information filled with the mode of that column
3. Text columns
    - Drop columns with any missing information

In [ ]:
#Preview current dataframe to allow confirmation that data is being dropped
display(housing_data)

In [ ]:
#1: Drop columns if > 5% data missing
def drop_missing(df):
    #Count number of missing values in each column
    missing_data = df.isnull().sum()
    
    #5% cutoff length
    cut_off = len(df)*0.05
    
    #Create df with columns containing <5% missing data
    dropped_cols = df.drop(missing_data[missing_data > cut_off].index, axis = 1)
    
    #Return cleaned df
    return dropped_cols

missing_dropped = drop_missing(housing_data)

#Display updated dataset to check number of cols
display(missing_dropped)

The number of columns has reduced from 82 to 71 so the first step has worked

In [ ]:
#2: Fill missing values for numerical columns if < 5% data missing
#Note all columns with >5% missing have already been removed above
def fill_na(df):
    #Identify numerical columns
    numeric_cols = df.select_dtypes(include = np.number)
   
    #Count missing values in each col and identify columns to fix
    count_missing = numeric_cols.isnull().sum()
    to_fix = count_missing[count_missing > 0]

    #Dictionary of modes for numerical columns that have na values    
    replacements = df[to_fix.index].mode().to_dict(orient = 'records')[0]
    
    dropped_na = df.fillna(replacements)
    
    return dropped_na

missing_numeric = missing_dropped.select_dtypes(include = ['int', 'float']).isnull().sum()
missing_numericals = missing_numeric[missing_numeric > 0]
print("Before removing null values:")
print(missing_numericals)

fixed_numeric = fill_na(missing_dropped)
fixed_numbers = fixed_numeric.select_dtypes(include = ['int', 'float']).isnull().sum()
fixed_numericals = fixed_numbers[fixed_numbers > 0]
print("\nAfter removing null values:")
print(fixed_numericals)

No numerical columns contain missing data after running the above function. We can now proceed to #3 of the process outlined above

In [ ]:
#3: Drop columns with missing text data
def drop_text(df):
    count_text_data = df.select_dtypes(include = ['object']).isnull().sum()
    count_text_data.head(10)
    
    missing_text = count_text_data[count_text_data > 0]
    missing_text_index = missing_text.index
    missing_text_dropped = df.drop(missing_text_index, axis = 1)
    
    return missing_text, missing_text_dropped
        
dropped_cols, cleaned_df = drop_text(fixed_numeric)
print("Columns that were dropped due to missing text information:\n",dropped_cols)
print("=================================================")
print("Cleaned dataframe:")
display(cleaned_df)

The dataframe has been reduced from 71 to 64 columns by removing text columns with missing data. The columns that were dropped are listed above.

## 2. Drop columns that aren't useful for ML model

After reviewing the [columns](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt) contained in the dataset, it is clear that some columns will not add any value to the ML model. These columns are listed below and removed in the following cell.


Column|Description
---|---
Order|Observation Number
PID|Parcel Identification Number

In [ ]:
non_ml_cols = ['Order','PID']
non_ml_removed = cleaned_df.drop(non_ml_cols, axis = 1)

#print preview of df to confirm columns were dropped. Number of cols should be 2 less than df above
display(non_ml_removed)

## 3. Feature engineering
New columns will now be created to quickly identify some information about the houses which is not immediately clear in the dataset. This includes identifying how old the house was when it was sold and how many years passed after the house was renovated before it was sold.

In [ ]:
#Create new columns
non_ml_removed['Age At Sale'] = non_ml_removed['Yr Sold'] - non_ml_removed['Year Built']
non_ml_removed['Yrs Between Remod & Sale'] = non_ml_removed['Yr Sold'] - non_ml_removed['Year Remod/Add']

#Check for values less than 0 in new columns (these represent errors):
display(non_ml_removed[non_ml_removed['Age At Sale'] < 0])
display(non_ml_removed[non_ml_removed['Yrs Between Remod & Sale'] < 0])


The rows shown above have some sort of error since the house could not have been sold before it was built or renovated. These rows will therefore be dropped from the dataset.

In [ ]:
cleaned_non_ml_removed = non_ml_removed.drop([1702, 2180, 2181])

## 4. Drop columns that leak final price information
Columns that may leak information were also determined by reviewing the columns of the dataset on the same webpage listed above. These columns are removed below.


In [ ]:
drop_leak_cols = ['Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold']
leaked_dropped = cleaned_non_ml_removed.drop(drop_leak_cols, axis = 1)

display(leaked_dropped)

# Linear Regression Model Development
The data cleaning and feature engineering strategy was determined above and can now be implemented into functions that execute all of the operationstested above. The dataset will then be split into test and training sets which will be used to train a linear regression model.

In [ ]:
#Define transform_features() to execture cleaning operations 1 to 4 from above
def transform_features(df):
    
    #Create df with <5% missing values in each column
    missing_data = df.isnull().sum()
    df = df.drop(missing_data[missing_data > len(df)*0.05].index, axis = 1)
    
    #Fill missing values for numerical columns
    num_missing = df.select_dtypes(include = np.number).isnull().sum()
    num_to_fix = num_missing[num_missing > 0]
    replacements = df[num_to_fix.index].mode().to_dict(orient = 'records')[0]
    df = df.fillna(replacements)
    
    #Remove text columns if any values missing
    count_text_data = df.select_dtypes(include = ['object']).isnull().sum()    
    missing_text = count_text_data[count_text_data > 0]
    df = df.drop(missing_text.index, axis = 1)
    
    #Create new columns
    df['Age At Sale'] = df['Yr Sold'] - df['Year Built']
    df['Yrs Between Remod & Sale'] = df['Yr Sold'] - df['Year Remod/Add']
    
    
    #Drop non-ml and columns and columns that leak price info
    drop_cols = ['Order','PID','Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold']
    df = df.drop(drop_cols, axis = 1)
    
    #Drop columns that were found to have false values after feature engineering
    df = df.drop([1702, 2180, 2181])
    
    #Return cleaned and engineered dataset
    return df


In [ ]:
#Function to extract training set
def get_datasets(df,length):
    #Create new train and test dataframes to keep original df in tact
    train = df.iloc[0:length]
    test = df.iloc[length:]
    return train, test

#Function to select features for training
#This is just a placeholder at this stage. To be developed in next section
def select_features(df, columns):
    return df[columns]

In [ ]:
#Function to train and test a LR model
def train_and_test(df):
    #Length of training dataset
    training_length = 1460
    #Isolate training and testing datasets
    train, test = get_datasets(df,training_length)
    
    #Numericalcolumns to use for training model
    numeric_train = train.select_dtypes(include = np.number)
    train_features = numeric_train.columns.drop('SalePrice')
    numeric_test = test.select_dtypes(include = np.number)
    test_features = numeric_test.columns.drop('SalePrice')
    
    #Dataframes of taining, target and test data
    train_data = select_features(train,train_features)
    train_target = select_features(train,['SalePrice'])
    test_data = select_features(test,test_features)

    #Fit linear regression model
    lr = LinearRegression()
    lr.fit(train_data,train_target)
    #Predict target column using model trained on selected columns
    predictions = lr.predict(test_data)
    
    #Calculate mse and rmse
    mse = mean_squared_error(test['SalePrice'], predictions)
    rmse = np.sqrt(mse)
    
    return rmse
    
#Clean and transform columns in dataset    
transformed_df = transform_features(housing_data)

#Get RMSE value for cleaned dataset using all columns to predict value
rmse = train_and_test(transformed_df)
print(rmse)

Now that the dataset has been cleaned and transformed and a model has been made to determined the RMSE value for a given set of columns, its time to move on to feature selection. This will identify the best columns to use for predicting the sale price and will be done in two steps which are described below

1. The correlation between each numerical column and the target column (SalePrice) will be assessed with a correlation matrix.
2. Columns that should be encoded as categorical will be converted into the correct type using [dummy variables](https://en.wikipedia.org/wiki/Dummy_variable_%28statistics%29)

# Feature Selection
---
## 1. Correlation
The correlation between each feature and the target column is assessed below

In [ ]:
#Specify target values
target = transformed_df['SalePrice']

#Extract only numerical columns
numerical_cols = transformed_df.select_dtypes(include = [np.number])

#Check correlation of each feature with SalePrice column
correlation_mat = numerical_cols.corr()['SalePrice'].sort_values()

#print correlation matrix
with pd.option_context('display.max_rows', 40):
    display(correlation_mat)

Negative values in the cell above indicate a negative correlation with sale price. This means that 'Age at Sale' has a strong negative correlation with the sale price. This makes intuitive sense since it would be expected that an older house would be sold for a lower price.

This information can be used to drop columns with correlation coefficients below a selected value.

## 2. Convert Categorical Columns
The features that should be categorical are identified and handled below

In [ ]:
#Columns that should be categorical from original dataset:
all_cat_cols = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
            "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
            "Exterior 2nd", "Exter Cond", "Exter Qual",  "Mas Vnr Type", "Foundation", "Functional", "Heating", 
                "Central Air", "Garage Type", "Misc Feature", "Sale Type", "Sale Condition", "Heating QC",
                "Kitchen Qual", "Land Slope", "Lot Shape", "Paved Drive", "Utilities"]

#Categorical columns that are still in cleaned, transformed dataset:
cat_cols = []
for col in all_cat_cols:
    if col in transformed_df.columns:
        cat_cols.append(col)

#Check categorical columns in dataset as determined above
cat_cols.sort()
print(cat_cols,'\n')

#Confirm categorical columns by comparing the list generated above with 'object' columns
text_cols = transformed_df.select_dtypes(include = ['object']).columns.tolist()
print(sorted(text_cols))

The columns shown above should contain categorical values. The 'MS SubClass' column was not picked up in the test at the end of the cell block since this column contains numerical values that are used to categorize the type of dwelling involved. This column should therefore still be included as a 'category' column.

Dummy coding will now be used below to separate each of these columns into n binary columns where n is the number of unique values in the column. 

Doing this for all columns irrespective of the number of unique values could make a massive dataframe so lets first check the number of unique values in each of these columns. Dummy coding will be used for columns that contain fewer than 10 unique values. Columns with more than 10 unique values will be removed to avoid making the dataframe too large.

In [ ]:
# Check for number of unique values in each categorical column
unique_vals = transformed_df[cat_cols].apply(lambda col: len(col.value_counts())).sort_values()

# Extract index of categorical columns with less than 10 unique values and remove columns with over 10 unique vals
unique_under_10 = unique_vals[unique_vals < 10].index
unique_over_10 = unique_vals[unique_vals >= 10].index
reduced_df = transformed_df.drop(unique_over_10, axis = 1)

# Convert categorical columns to 'category' datatype for use with get_dummies function
cat_cols = transformed_df[unique_under_10].copy()
for col in cat_cols:
    cat_cols[col] = cat_cols[col].astype('category')

# Create df with numiercal columns and dummy columns in place of categorical columns
df_with_cats = pd.concat([transformed_df, pd.get_dummies(cat_cols)], axis = 1)
df_with_cats = df_with_cats.drop(unique_under_10, axis = 1)

The cut-off number of unique feature values was chosen as 10 in the example above but can be pre-defined to avoid the dataset becoming too large.

# Linear Regression Model
The linear regression model from the section above can now be updated to include the feature selection techniques tested above. The LR model from above is copy-pasted below with the changes incorporated into the select_features function. 

The parameter 'k' is also introducted to the train_and_test function for k-fold cross validation. This requires if statements to be added to the function which alter the behaviour depending on the value of k since k-fold cross validation does not work for k = 0 ([holdout validation](https://medium.com/@eijaz/holdout-vs-cross-validation-in-machine-learning-7637112d3f8f)) or k = 1 (simple cross validation)

In [ ]:
#Define transform_features() to execture cleaning operations 1 to 4 from above
def transform_features(df):
    
    #Create df with <5% missing values in each column
    missing_data = df.isnull().sum()
    df = df.drop(missing_data[missing_data > len(df)*0.05].index, axis = 1)
    
    #Fill missing values for numerical columns
    num_missing = df.select_dtypes(include = np.number).isnull().sum()
    num_to_fix = num_missing[num_missing > 0]
    replacements = df[num_to_fix.index].mode().to_dict(orient = 'records')[0]
    df = df.fillna(replacements)
    
    #Remove text columns if any values missing
    count_text_data = df.select_dtypes(include = ['object']).isnull().sum()    
    missing_text = count_text_data[count_text_data > 0]
    df = df.drop(missing_text.index, axis = 1)
    
    #Create new columns
    df['Age At Sale'] = df['Yr Sold'] - df['Year Built']
    df['Yrs Between Remod & Sale'] = df['Yr Sold'] - df['Year Remod/Add']
    
    
    #Drop non-ml and columns and columns that leak price info
    drop_cols = ['Order','PID','Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold']
    df = df.drop(drop_cols, axis = 1)
    
    #Drop columns that were found to have false values after feature engineering
    df = df.drop([1702, 2180, 2181])
    
    #Return cleaned and engineered dataset
    return df

In [ ]:
#Function to extract training set
def get_datasets(df,length):
    #Create new train and test dataframes to keep original df in tact
    train = df.iloc[0:length]
    test = df.iloc[length:]
    return train, test

#Function to select features for training
def select_features(df, corr_cutoff, dummy_cutoff):

#Drop numerical features with correlation less than corr_cutoff
    target = df['SalePrice']

    #Extract only numerical columns
    numerical_cols = df.select_dtypes(include = [np.number])

    #Check correlation of each feature with SalePrice column
    corr_mat = numerical_cols.corr()['SalePrice'].sort_values()
    
    #Drop values with correlation lower than corr_cutoff
    df = df.drop(corr_mat[corr_mat < corr_cutoff].index, axis = 1)


# Convert categorical columns to dummy columns after dropping those with more than dummy_cutoff unique values

    #Columns that should be categorical from original dataset:
    all_cat_cols = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Exter Cond", "Exter Qual",  "Mas Vnr Type", "Foundation", "Functional", "Heating", 
                    "Central Air", "Garage Type", "Misc Feature", "Sale Type", "Sale Condition", "Heating QC",
                    "Kitchen Qual", "Land Slope", "Lot Shape", "Paved Drive", "Utilities"]

    #Categorical columns that are still in cleaned, transformed dataset:
    cat_cols = []
    for col in all_cat_cols:
        if col in df.columns:
            cat_cols.append(col)

    # Check for number of unique values in each categorical column
    unique_vals = df[cat_cols].apply(lambda col: len(col.value_counts())).sort_values()

    # Extract index of categorical columns with less than 10 unique values and remove columns with over 10 unique vals
    unique_under_cutoff = unique_vals[unique_vals < dummy_cutoff].index
    unique_over_cutoff = unique_vals[unique_vals >= dummy_cutoff].index
    reduced_df = df.drop(unique_over_cutoff, axis = 1)

    # Convert categorical columns to 'category' datatype for use with get_dummies function
    cat_cols = transformed_df[unique_under_10].copy()
    for col in cat_cols:
        cat_cols[col] = cat_cols[col].astype('category')

    # Create df with numerical columns and dummy columns with original categorical columns removed    df_with_cats = pd.concat([transformed_df, pd.get_dummies(cat_cols)], axis = 1)
    df_final = df_with_cats.drop(unique_over_cutoff, axis = 1)

    return df_final

In [ ]:
#Function to train and test a LR model. k = 0 by default
def train_and_test(df, k=0 ):
    #Isolate numeric features:
    numeric_features = df.select_dtypes(include = [np.number])
    features = numeric_features.columns.drop('SalePrice')
    lr = LinearRegression()

    #holdout validation
    if k == 0:
        #Length of training dataset
        training_length = 1460
        #Isolate training and testing datasets
        train, test = get_datasets(df,training_length)

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return predictions, rmse
    
    if k == 1:
         # Randomize all rows ie frac = 1
        shuffled_df = df.sample(frac = 1)
        #Length of training dataset
        training_length = 1460
        #Isolate training and testing datasets
        train, test = get_datasets(df,training_length)
        
        #First model - train on first half and test on seconds half
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        #Second model - train on seconds half and train on first half
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print("RMSE training on first half:",rmse_one)
        print("RMSE training on seconds half:",rmse_two)
        print("Average RMSE:", avg_rmse)
        return predictions_one, avg_rmse
    
    else:
        #Kfold cross validation using n splits
        #Use random_state argument for reproducible results
        kf = KFold(n_splits = k, shuffle = True, random_state = 3)
        rmse_values = []
        for train_index, test_index in kf.split(df):
            #Length of training dataset
            training_length = 1460

            #Dataframes of taining, target and test data
            train_data = df.iloc[train_index]
            test_data = df.iloc[test_index]

            #Fit linear regression model
            lr = LinearRegression()
            lr.fit(train_data[features],train_data['SalePrice'])
            #Predict target column using model trained on selected columns
            predictions = lr.predict(test_data[features])

            #Calculate mse and rmse
            mse = mean_squared_error(test_data['SalePrice'], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
            
        #Print actual vs predicted value
        print('Preview of actual prices vs predicted results')
        rounded = [elem.astype('int') for elem in predictions]
        data = {"SalePrice": test_data['SalePrice'], "Predicted Value":rounded}
        results = pd.DataFrame(data)
        display(results)
        
        #Print rmse values and plot results
        print("RMSE Values for K-fold cross validation:")
        print(rmse_values)
        fig = plt.figure(figsize = (8,8))
        plt.scatter(x=results['SalePrice'], y = results['Predicted Value'])
        plt.yticks(np.arange(100000,1000000,100000))
        plt.xlabel('Actual Sale Price', fontsize = 14)
        plt.ylabel('Predicted Sale Price', fontsize = 14)
        plt.title('Actual vs Predicted Sale Price', fontsize = 18)
            
        return predictions, np.mean(rmse_values)

In [ ]:
#Run transform_features() to clean dataset
transformed_df = transform_features(housing_data)

#Run select_features() to perform feature selection
filtered_df = select_features(transformed_df, 0.4, 10)

#Train and test KNN model
values, rmse = train_and_test(filtered_df, k = 4)

print("length",len(filtered_df))

In [ ]:
print("=================================================")
print("Average RMSE Value:",int(rmse))

# Conclusion
Linear regression was used to predict the sale price of houses in Ames, Iowa, USA. 

The operations to perform before training a linear regression model were first tested by dropping/filling missing data, dropping columns that would not be valuable to the model, performing feature engineering to produce new, meaningful columns and dropping columns that would reveal the final sale price of the house.

The linear regression model was then developed using K-Fold cross validation to estimate the skill of the model.

The cleaned dataset of 2927 rows was split into training and testing sets with the training set containing 1460 rows and the testing set containing the remainder.

The actual vs predicted sale price are shown in the chart below for a linear regression model using 4-fold cross validation 

<img src = "results.jpg">

The scatter plot above shows strongly linear results which indicate that the model accurately predicts the sale price of houses in Ames using the selected features. The average RMSE value for this model was 28 447.